<a href="https://colab.research.google.com/github/patilganesh1207/python-posting-blogger-wordpress/blob/main/phonepe_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import requests
import hashlib
import base64
import json
import datetime

MERCHANT_ID = 'M2306160483220675579140' #'PGTESTPAYUAT'
STORE_ID = 'MS220923145851009366116578'
TERMINAL_ID = 'MST220923150506572993581778'

API_KEYS = {}

QRINIT_ENDPOINT = "/v3/qr/init"
CHARGE_ENDPOINT = "/v3/charge"
TRANSACTION_ENDPOINT = "/v3/transaction"
REFUND_ENDPOINT = "/v3/credit/backToSource"
LINKINIT_ENDPOINT = "/v3/payLink/init"



def set_environment(ENV):
    global BASE_URL
    if ENV == "UAT":
        BASE_URL = "https://mercury-uat.phonepe.com"
    else:
        BASE_URL = "https://mercury.phonepe.com"


def make_base64(json_obj):
    json_str = json.dumps(json_obj, separators=(',', ':'))  # compact encoding
    return base64.urlsafe_b64encode(bytes(json_str, "utf-8")).decode("utf-8")


def make_hash(input_str):
    m = hashlib.sha256()
    m.update(input_str.encode())
    return m.hexdigest()


def make_request_body(base64_payload):
    request_body = {
        "request": base64_payload
    }
    data_json = json.dumps(request_body)
    return data_json


def make_charge_request(amount, transaction_id, mobile_number, salt_key_index):
    request_payload = {
        "amount": amount,  # Amount in paise
        "expiresIn": 180,
        "instrumentReference": mobile_number,
        "instrumentType": "MOBILE",
        "merchantId": MERCHANT_ID,
        "merchantOrderId": transaction_id,
        "storeId": STORE_ID,
        "terminalId": TERMINAL_ID,
        "transactionId": transaction_id,
        "message": "Payment for " + transaction_id
    }

    base64_payload = make_base64(request_payload)
    verification_str = base64_payload + CHARGE_ENDPOINT + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + CHARGE_ENDPOINT
    data = make_request_body(base64_payload)
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY
    }

    response = requests.request("POST", url, data=data, headers=headers)
    print(response.status_code, response.text)
    return response


def make_qrinit_request(amount, transaction_id, salt_key_index, callback, redirect):
    request_payload = {
        "amount": amount,  # Amount in paise
        "expiresIn": 180,
        "merchantId": MERCHANT_ID,
        "merchantOrderId": transaction_id,
        "storeId": STORE_ID,
        "terminalId": TERMINAL_ID,
        "transactionId": transaction_id,
        "message": "Payment for " + transaction_id
    }

    base64_payload = make_base64(request_payload)
    verification_str = base64_payload + QRINIT_ENDPOINT + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + QRINIT_ENDPOINT
    data = make_request_body(base64_payload)
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY,
        "X-CALLBACK-URL": callback,
        "X-CALL-MODE": "HTTP"
    }

    response = requests.request("POST", url, data=data, headers=headers)
    print(response.status_code, response.text)
    return response



def make_linkinit_request(amount, transaction_id, salt_key_index, callback, redirect):
    request_payload = {
        "amount": amount,  # Amount in paise
        "expiresIn": 180,
        "merchantId": MERCHANT_ID,
        "merchantOrderId": transaction_id,
        "storeId": STORE_ID,
        "terminalId": TERMINAL_ID,
        "transactionId": transaction_id,
        "message": "Payment for " + transaction_id
    }
    if amount == "edit" or amount == "":
        request_payload.pop('amount')

    base64_payload = make_base64(request_payload)
    verification_str = base64_payload + LINKINIT_ENDPOINT + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + LINKINIT_ENDPOINT
    data = make_request_body(base64_payload)
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY,
        "X-CALLBACK-URL": callback,
        "X-CALL-MODE": "HTTP"
    }

    response = requests.request("POST", url, data=data, headers=headers)
  #  print(response.status_code, response.text)
    return response


def make_status_request(transaction_id, salt_key_index):
    endpoint = TRANSACTION_ENDPOINT + "/" + MERCHANT_ID + "/" + transaction_id + "/status"
    verification_str = endpoint + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + endpoint
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY
    }

    response = requests.request("GET", url, headers=headers)
    print(response.status_code, response.text)
    return response


def make_cancel_request(transaction_id, salt_key_index):
    endpoint = CHARGE_ENDPOINT + "/" + MERCHANT_ID + "/" + transaction_id + "/cancel"
    verification_str = endpoint + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + endpoint
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY
    }

    response = requests.request("POST", url, headers=headers)
    print(response.status_code, response.text)
    return response


def make_refund_request(transaction_id, provider_reference_id, salt_key_index):
    request_payload = {
        "amount": 100,
        "merchantId": MERCHANT_ID,
        "providerReferenceId": provider_reference_id,
        "transactionId": transaction_id + "_refund",
        "message": "Refund"
    }

    base64_payload = make_base64(request_payload)
    verification_str = base64_payload + \
        REFUND_ENDPOINT + API_KEYS[salt_key_index]
    X_VERIFY = make_hash(verification_str) + "###" + salt_key_index

    url = BASE_URL + REFUND_ENDPOINT
    data = make_request_body(base64_payload)
    headers = {
        "Content-Type": "application/json",
        "X-VERIFY": X_VERIFY
    }

    response = requests.request("POST", url, data=data, headers=headers)
    print(response.status_code, response.text)
    return response
    
    
    
    

env = 'UAT' # Choose the environment first between UAT / Production

set_environment(env) # Possible values are UAT or PROD

""" Your credentials go here. 
You can get the UAT credentials from here : 
https://developer.phonepe.com/docs/test-credentials
You can get the production credentials by following the link below :
https://developer.phonepe.com/docs/merchant-onboarding
"""


api_keys = {
    '1': '8289e078-be0b-484d-ae60-052f117f8deb',
 #   '1': '099eb0cd-02cf-4e2a-8aca-3e6c6aff0399',
#   '1': 'a6334ff7-da0e-4d51-a9ce-76b97d518b1e',
    '2': '391510a8-af5c-4f97-9821-e330511a8840'
}

API_KEYS.update(api_keys)

customer_mobile_number = '7066229184'
salt_key_index = '1'



def main(ob):
    new_transaction_id = str(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))  # Setting the current date and time as orderId, for example
    amount = str(1)   # amount fixed fo 5 rs
    qr_id = str(ob['qr_id'])
    callback = "https://eu-gb.functions.appdomain.cloud/api/v1/web/patilganesh1207%40gmail.com_dev/default/et_sys_payment_eventpass/open-"+qr_id+"-"+amount+"-"+new_transaction_id
    redirect = "https://stores-moraya.web.app"
    
    response = make_linkinit_request(int(amount), new_transaction_id, salt_key_index, callback, redirect)
#response = make_charge_request(100, new_transaction_id, customer_mobile_number, salt_key_index)
#response_content = json.loads(response.content)
#if response_content['success'] is True:
#    providerReferenceId = json.loads(response.content)['data']['providerReferenceId']

#response = phonepe_api.make_status_request(new_transaction_id, salt_key_index)

#response = phonepe_api.make_cancel_request(new_transaction_id,salt_key_index)
#response = phonepe_api.make_refund_request(new_transaction_id, providerReferenceId, salt_key_index)
#response = make_status_request("T2209231459539753329033", salt_key_index)


    response = json.loads(response.text)
    print(response['data']['payLink'])
    
    p = """<div> <a style=" width: 200px; background-color: #1C2FFF; text-align: center; font-weight: 800; padding: 11px 0px; color: white; font-size: 12px; display: inline-block; text-decoration: none; " href='https://pmny.in/ZI1kxXryvd4R' > Pay Now </a> </div>"""

    return {
        'statusCode': 200,#301,
        'headers': {
      #      'Content-Type': 'text/plain',  # CORS ENABLED ONLY AT text/plain
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
      #      'Set-Cookie': '__com=12344;Max-Age=36000000;',
            'Cache-Control': 'no-cache, no-store, must-revalidate',
            'Pragma': 'no-cache', 
            'Expires': '0',
            'Location': response['data']['payLink'],
        },
        'body': p,
    }
main({"qr_id":"1"})

http://preprod.phon.pe/kj6oiloc


{'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': 'Content-Type',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
  'Cache-Control': 'no-cache, no-store, must-revalidate',
  'Pragma': 'no-cache',
  'Expires': '0',
  'Location': 'http://preprod.phon.pe/kj6oiloc'},
 'body': '<div> <a style=" width: 200px; background-color: #1C2FFF; text-align: center; font-weight: 800; padding: 11px 0px; color: white; font-size: 12px; display: inline-block; text-decoration: none; " href=\'https://pmny.in/ZI1kxXryvd4R\' > Pay Now </a> </div>'}